In [3]:
import sys
import pandas as pd
import numpy as np
import nltk
import os
import glob

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
from modules.load_data import load_data, DocID
from modules.predict import predict

In [6]:
df = load_data()

In [7]:
df

,author,message
0,Q,"\n\nHowever, there are frequent situations whe..."
1,K1,Download\n\nSource\n\nPDF\nActions\n Copy Pr...
2,K2,\n\nWith the rapid growth of the information c...


In [8]:
from modules.load_reference import load_reference

In [9]:
df_ref = load_reference(name="Brown")
df = pd.concat([df, df_ref])
df = df.reset_index(drop=True)
df

AHJAJAA


,author,message
0,Q,"\n\nHowever, there are frequent situations whe..."
1,K1,Download\n\nSource\n\nPDF\nActions\n Copy Pr...
2,K2,\n\nWith the rapid growth of the information c...
3,R,"Appointment of William S. Pfaff Jr. , 41 , as ..."


In [10]:
def tokenizeFunc(documents):
    # tf: Term frequency
    tf_vectorizer = CountVectorizer()
    tf_vectors = tf_vectorizer.fit_transform(documents)
    words = tf_vectorizer.get_feature_names_out()
    del tf_vectorizer
    return (tf_vectors, words)

In [11]:
documents = list(df['message'])
tf_vectors, words = tokenizeFunc(documents)


In [12]:
# len(words)
words[2360]

'talk'

In [13]:
tf_mat = tf_vectors.toarray()
del tf_vectors
df['tf'] = tf_mat.tolist()

In [14]:
df

,author,message,tf
0,Q,"\n\nHowever, there are frequent situations whe...","[0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, ..."
1,K1,Download\n\nSource\n\nPDF\nActions\n Copy Pr...,"[0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, ..."
2,K2,\n\nWith the rapid growth of the information c...,"[2, 1, 1, 7, 22, 3, 2, 4, 1, 13, 4, 1, 6, 1, 0..."
3,R,"Appointment of William S. Pfaff Jr. , 41 , as ...","[1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2, ..."


# Predict by Word frequency

In [15]:
from modules.predict import predict
Q_df = df.loc[DocID.Q]
K_df = df.loc[DocID.K1:DocID.K2].reset_index(drop=True)

In [16]:
predict(Q_df, K_df, target='tf', _list=words)

==================== Top 20 ====================
Suspected : {'K2', 'K1'}
Q  :the | of | and | to | for | in | be | or | planning | are | is | as | time | location | user | based | that | system | soft | on | 


K1 : the | of | and | in | to | as | music | is | for | be | on | by | tchaikovsky | can | that | not | between | similarity | such | models | 
Score : 11
Shared list Q-K1 : in | the | as | for | of | to | is | on | and | that | be | 

K2 : the | of | to | and | in | is | high | eo | with | for | temperature | at | gbit | modulator | device | are | as | sph | polymer | up | 
Score : 9
Shared list Q-K2 : in | as | for | of | to | is | and | are | the | 
------------------------------------------------
==================== Top 40 ====================
Suspected : {'K2', 'K1'}
Q  :the | of | and | to | for | in | be | or | planning | are | is | as | time | location | user | based | that | system | soft | on | such | figure | by | activity | with | it | scenarios | not | reminders |

'K1'

# Predict by Keyword Frequency
First calucurate the keyness values for each normalized vector

In [17]:
from modules.keyness import keyness, create_keyness_col

In [18]:
create_keyness_col(df, effort='%diff')

In [19]:
df

,author,message,tf,keyness
0,Q,"\n\nHowever, there are frequent situations whe...","[0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, ...","[-99.99999931273996, -31.273996509598604, -31...."
1,K1,Download\n\nSource\n\nPDF\nActions\n Copy Pr...,"[0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, ...","[-99.9999987205978, 27.94022092267706, 27.9402..."
2,K2,\n\nWith the rapid growth of the information c...,"[2, 1, 1, 7, 22, 3, 2, 4, 1, 13, 4, 1, 6, 1, 0...","[-19.270192701927016, 4036490264.903649, 40364..."
3,R,"Appointment of William S. Pfaff Jr. , 41 , as ...","[1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [20]:
Q_df = df.loc[DocID.Q]
K_df = df.loc[DocID.K1:DocID.K2].reset_index(drop=True)

In [21]:
predict(Q_df, K_df, target='keyness', _list=words)

==================== Top 20 ====================
Suspected : {'K2', 'K1'}
Q  :planning | location | user | based | system | soft | figure | such | activity | scenarios | reminders | we | notifications | reminder | ontology | notification | helpful | project | example | section | 


K1 : music | tchaikovsky | similarity | such | models | speac | figure | children | musicology | album | schumann | harmonic | art | manuscript | based | representation | analysis | style | transformations | published | 
Score : 3
Shared list Q-K1 : such | figure | based | 

K2 : eo | temperature | gbit | modulator | device | sph | polymer | energy | speed | thermal | operating | 100 | signals | silicon | fig | ambient | 200 | ultra | bandwidth | modulators | 
Score : 0
Shared list Q-K2 : 
------------------------------------------------
==================== Top 40 ====================
Suspected : {'K2', 'K1'}
Q  :planning | location | user | based | system | soft | figure | such | activity | scenarios | rem

'K1'